# Tabular data - 5-way classifier
# SHAP implementation
Code to incorporate SHAP has been added to the copy of the single run script

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from string import ascii_letters
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from sklearn.metrics import plot_confusion_matrix, log_loss
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.inspection import permutation_importance
from sklearn.datasets import make_classification  
from sklearn.neighbors import KNeighborsClassifier   
from sklearn.metrics import roc_curve
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import BayesianRidge
from sklearn.metrics import classification_report
from sklearn import svm
from sklearn.preprocessing import StandardScaler, PowerTransformer
import tensorflow as tf
from tensorflow.random import set_seed
set_seed(42)
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint
import pickle
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix, classification_report
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.models import Model
from sklearn.preprocessing import PowerTransformer
from scipy import optimize
import os

In [ ]:
img_dpi= 300

In [ ]:
path_res = Path("/project/restored-files/DiseaseClassifierPaper2021/chemPredPD/demo_output/tab_data_5way_SHAP")

# Set paths

## Results paths

In [ ]:
#Code to set the path to save the model
path_models = path_res.joinpath("saved_models")
print(path_models)
path_models.is_dir()

In [ ]:
#Code to set the path to save plots
path_plots = path_res.joinpath("plots")
print(path_plots)
path_plots.is_dir()

In [ ]:
#Code to create folders if not present
if not path_plots.is_dir():
    path_plots.mkdir(parents=True, exist_ok=True)
path_plots.is_dir()
if not path_models.exists():
    path_models.mkdir(parents=True, exist_ok=True)
path_models.is_dir()

## Load data

In [ ]:
df1 = pd.read_csv (r'/project/restored-files/DiseaseClassifierPaper2021/chemPredPD/demo_data/Tabular_subset.csv')

In [ ]:
df1.head()

In [ ]:
list(df1.columns)

In [ ]:
df1.shape

In [ ]:
df_features = df1.iloc[:, 1:41]

In [ ]:
df_features.head()

In [ ]:
df_features.shape

In [ ]:
for i, col in enumerate(df_features.columns):
    plt.figure(i)
    sns.distplot(df1[col])
    plt.show()

In [ ]:
corr = df_features.corr()
mask = np.triu(np.ones_like(corr, dtype=bool))
f, ax = plt.subplots(figsize=(35, 25))
cmap = sns.diverging_palette(230, 20, as_cmap=True)
sns.heatmap(corr, annot = True, fmt='.2f', mask=mask, cmap=cmap, vmax=1, vmin=-1, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

In [ ]:
df1["label"] = df1.label.astype('category')

In [ ]:
df1.insert(0, 'pathway', df1.label.cat.codes)

df1.head()

In [ ]:
all_cols = list(df1)
len(all_cols)

In [ ]:
all_feats = [i for i in all_cols if i.startswith('Live')]
len(all_feats)

In [ ]:
all_cols

In [ ]:
mask = df1[all_cols].isnull().sum(axis=1)

tmp = pd.DataFrame(
#     pd.concat([
    pd.DataFrame(mask[mask>0], columns=["number_of_points"])["number_of_points"].value_counts().reset_index().rename(
        columns={'index':'number_of_features_missing'}
    ).set_index("number_of_points"), 
)

tmp['percentage_feature_missing'] = np.round(tmp['number_of_features_missing']/len(all_cols)*100., decimals=2)
tmp

In [ ]:
mask = df1[all_feats].isnull().sum(axis=1)

to_remove = pd.DataFrame(mask[mask>=20], columns=["number_of_points"]) # these are the five points that need to be dropped.

to_remove

In [ ]:
df1.shape

In [ ]:
df1 = df1.drop(index=to_remove.index)
df1.shape

In [ ]:
plate_cols = ['PlateID']

In [ ]:
label_cols = ['label', 'label_name', 'pathway']

In [ ]:
df_train, df_test = train_test_split(df1,test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_train, test_size=0.2, random_state=42)

In [ ]:
class SafePowerTransformer(PowerTransformer):
    def _yeo_johnson_optimize(self, x):
        def _neg_log_likelihood(lmbda):
            x_trans = self._yeo_johnson_transform(x, lmbda)
            n_samples = x.shape[0]
            variance = x_trans.var()
            if variance == 0:
                return np.inf  
            loglike = -n_samples / 2 * np.log(variance) 
            loglike += (lmbda - 1) * (np.sign(x) * np.log1p(np.abs(x))).sum()
            return -loglike
        x = x[~np.isnan(x)]
        return optimize.brent(_neg_log_likelihood, brack=(-2, 2))

In [ ]:
imp = IterativeImputer(max_iter=10, random_state=0, initial_strategy='median')
lambdas = {}

scaled_df_train = []
for eachplate in df_train.PlateID.unique():
    dd = df_train[df_train.PlateID==eachplate].copy()
    
    print(f"plate:{eachplate}, shape:{dd.shape}")

    # fit for scaling. Scale per control per plate, then apply the transformation to all datapoints in the plate.
    scaler = SafePowerTransformer()
    scaler.fit(dd[dd.label==4][all_feats].values)

    # scale
    dd.loc[:,all_feats] = scaler.transform(dd[all_feats].values)

    # inpute missing values using iterative inputer
    dd.loc[:,all_feats] = imp.fit_transform(dd[all_feats].values)

    lambdas[eachplate] = scaler.lambdas_
    scaled_df_train.append(dd)
    
scaled_df_train = pd.concat(scaled_df_train)
print(scaled_df_train.shape)

scaled_df_train.head(10)

In [ ]:
scaled_df_test = []
for eachplate in df_test.PlateID.unique():
    dd = df_test[df_test.PlateID==eachplate].copy()
    
    print(f"plate:{eachplate}, shape:{dd.shape}")

    # fit for scaling. Scale per control per plate, then apply the transformation to all datapoints in the plate.
    scaler = SafePowerTransformer()
    scaler.fit(dd[dd.label==4][all_feats].values)

    # scale
    dd.loc[:,all_feats] = scaler.transform(dd[all_feats].values)

    # inpute missing values using iterative inputer
    dd.loc[:,all_feats] = imp.fit_transform(dd[all_feats].values)

    lambdas[eachplate] = scaler.lambdas_
    scaled_df_test.append(dd)
    
scaled_df_test = pd.concat(scaled_df_test)
print(scaled_df_test.shape)

scaled_df_test.head(10)

In [ ]:
scaled_df_val = []
for eachplate in df_val.PlateID.unique():
    dd = df_val[df_val.PlateID==eachplate].copy()
    
    print(f"plate:{eachplate}, shape:{dd.shape}")

    # fit for scaling. Scale per control per plate, then apply the transformation to all datapoints in the plate.
    scaler = SafePowerTransformer()
    scaler.fit(dd[dd.label==4][all_feats].values)

    # scale
    dd.loc[:,all_feats] = scaler.transform(dd[all_feats].values)

    # inpute missing values using iterative inputer
    dd.loc[:,all_feats] = imp.fit_transform(dd[all_feats].values)

    lambdas[eachplate] = scaler.lambdas_
    scaled_df_val.append(dd)
    
scaled_df_val = pd.concat(scaled_df_val)
print(scaled_df_val.shape)

scaled_df_val.head(10)

In [ ]:
print(scaled_df_train.isnull().sum().sum())
print(scaled_df_val.isnull().sum().sum())
print(scaled_df_test.isnull().sum().sum())

In [ ]:
print(scaled_df_train.shape)
print(scaled_df_val.shape)
print(scaled_df_test.shape)

In [ ]:
with np.printoptions(precision=2, suppress=True):
    print(np.array(list(lambdas.values())))

In [ ]:
np.array(list(lambdas.values()))[:,0].shape

In [ ]:
fig, ax = plt.subplots(figsize=(8,12))
sns.pointplot(data=pd.DataFrame(np.array(list(lambdas.values())), columns=all_feats), join=False, orient='h')

In [ ]:
lambdas_df = pd.DataFrame(np.array(list(lambdas.values())), columns=all_feats, index=list(lambdas.keys()))
lambdas_df.index.rename("Plate_ID", inplace=True)
lambdas_df

In [ ]:
fig, ax = plt.subplots(figsize=(8,12))
sns.pointplot(
    data=lambdas_df, 
    join=False, 
    orient='h',
    order=list(lambdas_df.std().sort_values(ascending=False).index),
)

ax.set_xlabel("Average value of scaling factor (with std error) for CONTROL distribution across plates ")

In [ ]:
#scaled_X_train.drop(scaled_X_train[scaled_X_train['label'] == 4].index, inplace = True)

In [ ]:
#scaled_X_test.drop(scaled_X_test[scaled_X_test['label'] == 4].index, inplace = True)

In [ ]:
scaled_df_train.PlateID.value_counts()/len(scaled_df_train)

In [ ]:
scaled_df_test.PlateID.value_counts()/len(scaled_df_test)

In [ ]:
scaled_df_val.PlateID.value_counts()/len(scaled_df_val)

In [ ]:
print(scaled_df_train.shape)
print(scaled_df_test.shape)
print(scaled_df_val.shape)

In [ ]:
scaled_dds_train = []
for eachp in scaled_df_train.PlateID.unique():
    dd = scaled_df_train[scaled_df_train.PlateID==eachp].sample(n=scaled_df_train.PlateID.value_counts().min())
    scaled_dds_train.append(dd)
    
scaled_dds_train = pd.concat(scaled_dds_train).sample(frac=1).reset_index(drop=True)
print(scaled_dds_train.shape, scaled_df_train.PlateID.value_counts().min()*scaled_df_train.PlateID.nunique())

# del scaled_dfs
scaled_dds_train.head(3)

In [ ]:
scaled_dds_val = []
for eachp in scaled_df_val.PlateID.unique():
    dd = scaled_df_val[scaled_df_val.PlateID==eachp].sample(n=scaled_df_val.PlateID.value_counts().min())
    scaled_dds_val.append(dd)
    
scaled_dds_val = pd.concat(scaled_dds_val).sample(frac=1).reset_index(drop=True)
print(scaled_dds_val.shape, scaled_df_val.PlateID.value_counts().min()*scaled_df_val.PlateID.nunique())

# del scaled_dfs
scaled_dds_val.head(3)

In [ ]:
scaled_dds_test = []
for eachp in scaled_df_test.PlateID.unique():
    dd = scaled_df_test[scaled_df_test.PlateID==eachp].sample(n=scaled_df_test.PlateID.value_counts().min())
    scaled_dds_test.append(dd)
    
scaled_dds_test = pd.concat(scaled_dds_test).sample(frac=1).reset_index(drop=True)
print(scaled_dds_test.shape, scaled_df_test.PlateID.value_counts().min()*scaled_df_test.PlateID.nunique())

# del scaled_dfs
scaled_dds_test.head(3)

In [ ]:
all_feats

In [ ]:
x_train = scaled_df_train[all_feats]
x_val = scaled_df_val[all_feats]
x_test = scaled_df_test[all_feats]

y_train = scaled_df_train['label'].values
y_val = scaled_df_val['label'].values
y_test = scaled_df_test['label'].values

In [ ]:
len(x_test)

In [ ]:
scal_y_train_onehot = to_categorical(y_train)
scal_y_train_onehot

In [ ]:
scal_y_val_onehot = to_categorical(y_val)
scal_y_val_onehot

In [ ]:
scal_y_test_onehot = to_categorical(y_test)
scal_y_test_onehot

In [ ]:
scal_y_test_onehot.shape[1]

In [ ]:
np.unique(y_train, return_counts=True)

In [ ]:
scal_model_name = "predicting_disease_pathway_all_conditions_powertrans_plate-scaling_dense-40_3"

In [ ]:
scal_checkpoint = ModelCheckpoint(
    str(path_models/scal_model_name)+'.hdf5', 
    monitor='val_loss', 
    verbose=1,
    save_best_only=True, 
    mode='auto', 
    save_freq='epoch'
)

In [ ]:
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', 
    min_delta=0, 
    patience=50, 
    verbose=0, 
    mode='auto',
    baseline=None, 
    restore_best_weights=True,
)

In [ ]:
dropout_rate = 0.1

In [ ]:
input_t = Input(shape=(x_train.shape[1],))

# x = Dense(10, activation='relu')(input_t)

x = Dense(40, activation='relu')(input_t)
x = Dropout(dropout_rate)(x)
x = Dense(40, activation='relu')(x)
x = Dropout(dropout_rate)(x)
x = Dense(40, activation='relu')(x)
x = Dropout(dropout_rate)(x)


output = Dense(scal_y_train_onehot.shape[1], activation='softmax')(x)

scal_model = Model(inputs=input_t, outputs = output)
scal_model.compile(
    loss='categorical_crossentropy',
    metrics=['accuracy'],
    optimizer='adam',
)

In [ ]:
batch_size = 100
epochs = 1000
history = scal_model.fit(
    x_train,
    scal_y_train_onehot,
    validation_data = (x_val, scal_y_val_onehot),
    batch_size = batch_size,
    epochs = epochs,
    callbacks=[scal_checkpoint, early_stop],
#     callbacks=[scal_checkpoint, early_stop],
)

# Plot model histories and scores

In [ ]:
scal_model_descr = ("Disease pathways and control\nFilling and "
               f"powerscaling per control within plate, model: {scal_model_name.split('_')[-1]}")

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(14,7))

ax1.plot(history.history['loss'], label='Training')
ax1.plot(history.history['val_loss'], label='Validation')
ax1.legend(frameon=False)
ax1.set_title("Loss")
ax1.set_ylabel("Loss")
ax1.set_xlabel("Epochs")

ax2.plot(history.history['accuracy'], label='Training')
ax2.plot(history.history['val_accuracy'], label='Validation')
ax2.legend(frameon=False)
ax2.set_title("Accuracy")
ax2.set_ylabel("Accuracy")
ax2.set_xlabel("Epochs")

fig.suptitle(scal_model_descr);
fig.savefig(path_plots.joinpath(f"Model-perform_{os.path.basename(scal_model_name)}.png"), bbox_inches='tight', dpi= img_dpi)

In [ ]:
scal_model_saved = tf.keras.models.load_model(str(path_models/scal_model_name)+'.hdf5')

In [ ]:
scal_y_pred = scal_model_saved.predict(x_test)

In [ ]:
df_train['label_name'] = pd.Categorical(df_train.label_name)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(50,20))

ax1.set_title("Count")
scal_matrix = confusion_matrix(
    scal_y_test_onehot.argmax(axis=1), scal_y_pred.argmax(axis=1)
)

scal_matrix_norm = confusion_matrix(
    scal_y_test_onehot.argmax(axis=1), scal_y_pred.argmax(axis=1),
    normalize='true',
)

ConfusionMatrixDisplay(
    confusion_matrix=scal_matrix,
    display_labels=['3xSNCA', 'Oligomer', ' Rotenone', 'Oligomycin/Antimycin', 'Control']
).plot(cmap='Blues', ax=ax1, values_format="g")

ax1.set_ylabel(ax1.get_ylabel(), fontsize=20)
ax1.set_xlabel(ax1.get_xlabel(), fontsize=20, labelpad=10)
fig.axes[-1].set_visible(False)

# Saving the counts and fraction separately using extent
extent = ax1.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
# Pad the saved area by 50% in the x-direction and 20% in the y-direction
fig.savefig(path_plots.joinpath(f"CONFMATRIX_count_exp_{os.path.basename(scal_model_name)}_Scale-controlplate.png"),
            bbox_inches=extent.expanded(1.5, 1.2), dpi=img_dpi, )


ax2.set_title("Fraction")
ConfusionMatrixDisplay(
    confusion_matrix=scal_matrix_norm,
    display_labels=['3xSNCA', 'Oligomer', ' Rotenone', 'Oligomycin/Antimycin', 'Control']
).plot(cmap='Blues', ax=ax2, values_format="g")

ax2.set_ylabel(ax2.get_ylabel(), fontsize=20)
ax2.set_xlabel(ax2.get_xlabel(), fontsize=20, labelpad=10)
fig.axes[-1].set_visible(False)

extent = ax2.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
# Pad the saved area in the x-direction and y-direction
fig.savefig(path_plots.joinpath(f"CONFMATRIX_fraction_exp_{os.path.basename(scal_model_name)}_Scale-controlplate.png"),
            bbox_inches=extent.expanded(1.5, 1.2), dpi=img_dpi)

fig.tight_layout()
fig.suptitle(f"CONFUSION MATRIX\n{scal_model_descr}", y=1.01)

plt.show()


print('\n\n', 
      classification_report(scal_y_test_onehot.argmax(axis=1), scal_y_pred.argmax(axis=1))
     )

# install and run SHAP

In [ ]:
!pip install shap

In [ ]:
import shap

In [ ]:
df_train_normed_summary = shap.kmeans(x_train, 25)

In [ ]:
explainer = shap.KernelExplainer(scal_model.predict, df_train_normed_summary)

In [ ]:
len(x_test)

In [ ]:
shap_values = explainer.shap_values(x_test)

In [ ]:
shap.summary_plot(shap_values[0], x_test, max_display=50, show=False)
plt.savefig(path_plots.joinpath(f"SHAP_SNCA_{os.path.basename(scal_model_name)}.png"), bbox_inches=extent.expanded(1.5, 1.2), dpi=img_dpi)

In [ ]:
shap.summary_plot(shap_values[1], x_test, max_display=50, show=False)
plt.savefig(path_plots.joinpath(f"SHAP_oligomer_{os.path.basename(scal_model_name)}.png"), bbox_inches=extent.expanded(1.5, 1.2), dpi=img_dpi)

In [ ]:
shap.summary_plot(shap_values[2], x_test, max_display=50, show=False)
plt.savefig(path_plots.joinpath(f"SHAP_rotenone_{os.path.basename(scal_model_name)}.png"), bbox_inches=extent.expanded(1.5, 1.2), dpi=img_dpi)

In [ ]:
shap.summary_plot(shap_values[3], x_test, max_display=50, show=False)
plt.savefig(path_plots.joinpath(f"SHAP_OA_{os.path.basename(scal_model_name)}.png"), bbox_inches=extent.expanded(1.5, 1.2), dpi=img_dpi)

In [ ]:
shap.summary_plot(shap_values[4], x_test, max_display=50, show=False)
plt.savefig(path_plots.joinpath(f"SHAP_control_{os.path.basename(scal_model_name)}.png"), bbox_inches=extent.expanded(1.5, 1.2), dpi=img_dpi)

In [ ]:
classes = np.array(["SNCAx3", "Oligomer", "Rotenone", "Oligomycin/Antimycin", "Control"])

In [ ]:
shap.summary_plot(shap_values, x_test, plot_type='bar', max_display=50, show=False, class_names=classes)
plt.savefig(path_plots.joinpath(f"SHAP_feature_importance_{os.path.basename(scal_model_name)}.png"), bbox_inches=extent.expanded(1.5, 1.2), dpi=img_dpi)

In [ ]:
shap.summary_plot(shap_values[0], x_test, max_display=10, show=False)
plt.savefig(path_plots.joinpath(f"SHAP_SNCA_10_{os.path.basename(scal_model_name)}.png"), bbox_inches=extent.expanded(1.5, 1.2), dpi=img_dpi)

In [ ]:
shap.summary_plot(shap_values[1], x_test, max_display=10, show=False)
plt.savefig(path_plots.joinpath(f"SHAP_oligomer_10_{os.path.basename(scal_model_name)}.png"), bbox_inches=extent.expanded(1.5, 1.2), dpi=img_dpi)

In [ ]:
shap.summary_plot(shap_values[2], x_test, max_display=10, show=False)
plt.savefig(path_plots.joinpath(f"SHAP_rotenone_10_{os.path.basename(scal_model_name)}.png"), bbox_inches=extent.expanded(1.5, 1.2), dpi=img_dpi)

In [ ]:
shap.summary_plot(shap_values[3], x_test, max_display=10, show=False)
plt.savefig(path_plots.joinpath(f"SHAP_OA_10_{os.path.basename(scal_model_name)}.png"), bbox_inches=extent.expanded(1.5, 1.2), dpi=img_dpi)

In [ ]:
shap.summary_plot(shap_values[4], x_test, max_display=10, show=False)
plt.savefig(path_plots.joinpath(f"SHAP_control_10_{os.path.basename(scal_model_name)}.png"), bbox_inches=extent.expanded(1.5, 1.2), dpi=img_dpi)